> ### Problem: Adversarial Noise (from the author's description)
>The task involves developing a program that manipulates images by adding adversarial noise. This noise is designed to trick an image classification model into misclassifying the altered image as a specified target class, regardless of the original content.
> - You may select any pre-trained image classification model for this task. A model from the torchvision library is recommended, but not mandatory.
> - The core challenge is to effectively introduce noise into the image in such a way that the model misclassifies it as the desired target class, without making the noise perceptible to a casual human viewer.





### 0.1 Getting Started

The goal of the challenge is to develop a program that adds adversarial noise to images with the aim of tricking an image classification model without being detected by a human. 

More formally, consider that $\mathcal{D} = \{ (x_i, y_i) \}_{i=1}^N$ is a dataset of images and their corresponding labels. Let also $f_{\theta}: \mathcal{X} \to \mathcal{Y}$ be a pre-trained image classification model with estimated parameters $\theta$, where $\mathcal{X}$ is the space of images and $\mathcal{Y}$ is the space of labels that achieves $a%$ accuracy a dataset. The goal is to find an adversarial noise $\epsilon$ such that the model misclassifies the image $x_i$:

$$
\epsilon^* = \arg\min_{\epsilon} \frac{1}{N} \sum_{i=1}^N \mathbb{I}\Big(f_{\theta}(x_i + \epsilon) = y_i\Big)
$$

Thus, we have the following model components:

1. **Dataset**: A collection of images and their corresponding labels.
2. **Model**: A pre-trained image classification model that can be used to classify images.
3. **Adversarial Noise**: A perturbation that can be added to the images

This will guide the development of the solution thorough this notebook.

### 0.2 Dataset

The dataset used in this challenge is the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset, which consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. The dataset is divided into 50,000 training images and 10,000 test images.

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Path to your CIFAR-10 data
data_path = os.path.join(os.getcwd(), "data", "inputs", "cifar-10-batches-py")

# Function to load a single batch
def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        data = batch[b'data']
        labels = batch[b'labels']
        # Reshape to (num_images, 3, 32, 32)
        data = data.reshape(-1, 3, 32, 32)
        return data, labels

# Load the first batch
X_batch, y_batch = load_cifar_batch(f"{data_path}/data_batch_1")

# Load label names
with open(f"{data_path}/batches.meta", 'rb') as f:
    label_names = pickle.load(f, encoding='bytes')[b'label_names']
    label_names = [name.decode('utf-8') for name in label_names]

# Pick a random image index
idx = np.random.randint(0, len(X_batch))
img = X_batch[idx]
label = label_names[y_batch[idx]]

# Transpose image to (32, 32, 3) for plotting
img = np.transpose(img, (1, 2, 0))

# Plot the image
plt.imshow(img)
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()
